In [5]:
from Bio.Blast import NCBIXML
import pandas as pd
from pandas.io.json import json_normalize

result = open("blast_result-ITS2.xml", "r")
records = NCBIXML.parse(result)

#Create the outer list
final = []
for blast_record in records:
    dic = {}
    #print(blast_record.query)
    query_title = blast_record.query.split("|")[0]
    dic["id"] = query_title
    pairs = []
    for i in blast_record.descriptions:
        ncbi = i.title.split("|")[3]
        name = " ".join(i.title.split("|")[4].split(" ")[1:3])
        e_value = i.e
        score = i.score
        dic_pairs = {}
        # Make a dictionary containing the name and the e_value
        dic_pairs["hit"] = name
        dic_pairs["e-value"] = e_value
        dic_pairs["ncbi"] = ncbi
        dic_pairs["score"] = score
        dic_pairs["gi_number"] = i.title.split("|")[1]
        pairs.append(dic_pairs)
        #Maybe use a counter to break out of the loop
    k =[]
    for alignment in blast_record.alignments:
       # gi_alignment = alignment.title.split("|")[1]
        #identity["id2"] = gi_alignment
        new = {}
        #print(alignment.hsps.align_length)
        for hsp in alignment.hsps:
            new["coverage"] = hsp.align_length / blast_record.query_length
            new["bits"] = hsp.bits
            new["percent"] = hsp.identities / hsp.align_length
            k.append(new)

    dic["pairs"] = pairs
    dic["k"] = k
    final.append(dic)


In [6]:
ndata_pairs = json_normalize(final, 'pairs', ['id'])
ndata_k = json_normalize(final, 'k', ['id'])
df_pairs=pd.DataFrame(ndata_pairs)
df_k = pd.DataFrame(ndata_k)

In [7]:
fainal_df = pd.merge(df_k, df_pairs, left_index=True, right_index=True)

In [8]:
fainal_df.head()

,id_x,id_y


In [9]:
df.hit.unique()

NameError: name 'df' is not defined